<a href="https://colab.research.google.com/github/allen-n/tasty-fonts/blob/master/FontExtraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Testing implementations for [Tasty Fonts](https://docs.google.com/document/d/169VBNGQaOpb_oGjEciiqwTElGoaIayObB6Cc3lqVBAU). 


In [1]:
# Do library imports
import os
import numpy as np  
import matplotlib.pyplot as plt
import tensorflow as tf
import glob
from tqdm.autonotebook import tqdm
import pathlib

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  import sys


In [2]:
# Set constants
AUTOTUNE = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 32
IMG_HEIGHT = 224
IMG_WIDTH = 224
SHUFFLE_BUFFER_SIZE = 1000

In [3]:

# Load remote font data as .png files
data_dir_name = "224"
data_dir_origin = "https://drive.google.com/u/0/uc?id=1OMlw5G2tpZtAwI9PyKU7DaU8WviaXBRI&export=download"
data_dir = tf.keras.utils.get_file(fname=data_dir_name, untar=True, 
                                  origin=data_dir_origin)
data_dir = pathlib.Path(data_dir)

In [4]:
# choose the local (colab) keras directory to read the data.
image_count = len(list(data_dir.glob('*.png')))
STEPS_PER_EPOCH = np.ceil(image_count/BATCH_SIZE)
print("{} Font images stored".format(image_count))


1883 Font images stored


In [5]:
list_ds = tf.data.Dataset.list_files(str(data_dir/'*'))
for f in tqdm(list_ds.take(5)):
  print(f.numpy())

b'/root/.keras/datasets/224/000466-font-82-700italic-Alegreya+Sans.png'
b'/root/.keras/datasets/224/001721-font-680-regular-Nova+Flat.png'
b'/root/.keras/datasets/224/001016-font-294-600-Hind+Guntur.png'
b'/root/.keras/datasets/224/001549-font-560-regular-Rozha+One.png'
b'/root/.keras/datasets/224/000414-font-73-500-Source+Code+Pro.png'



In [6]:
def get_label(file_path):
  # convert the path to a list of path components
  parts = tf.strings.split(file_path, os.path.sep)
  # The class name is just the name of the font file (i.e. image name)
  return parts[-1]

In [7]:
def decode_png(img):
  # convert the compressed string to a 3D uint8 tensor
  img = tf.image.decode_png(img, channels=3)
  # Use `convert_image_dtype` to convert to floats in the [0,1] range.
  img = tf.image.convert_image_dtype(img, tf.float32)
  # resize the image to the desired size.
  return tf.image.resize(img, [IMG_HEIGHT, IMG_WIDTH])

In [8]:
def process_path(file_path):
  label = get_label(file_path)
  # load the raw data from the file as a string
  img = tf.io.read_file(file_path)
  img = decode_png(img)
  return img, label

In [17]:
# Set `num_parallel_calls` so multiple images are loaded/processed in parallel.
labeled_ds = list_ds.map(process_path, num_parallel_calls=AUTOTUNE)

In [10]:
for image, label in labeled_ds.take(1):
  print("Image shape: ", image.numpy().shape)
  print("Label: ", label.numpy())
  print(type(image))

Image shape:  (224, 224, 3)
Label:  b'000811-font-208-700-Share.png'
<class 'tensorflow.python.framework.ops.EagerTensor'>


In [11]:
IMG_SHAPE = (IMG_HEIGHT, IMG_WIDTH, 3)

# Create the base model from the pre-trained model MobileNet V2
# Append average pooling layer for creating font vector
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet',
                                               pooling='avg') 

In [12]:
font_train_batches = labeled_ds.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)


## Check if everything looks correct


In [13]:
for image_batch, label_batch in font_train_batches.take(1):
  pass
print(type(image_batch))
print(image_batch.shape)



<class 'tensorflow.python.framework.ops.EagerTensor'>
(32, 224, 224, 3)


In [14]:
example_feature_batch = base_model(image_batch)
print(example_feature_batch.shape)

(32, 1280)


## Continue Processing

In [19]:
font_train_batches = labeled_ds.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)


In [20]:
base_model.trainable = False
# base_model.summary()


In [21]:
# Will throw an invalid argument error if the font train batches aren't 
# re-generated
font_vectors = []
font_labels = []

for image_batch, label_batch in tqdm(font_train_batches.take(2)):
  feature_batch = base_model(image_batch)
  font_vectors.append(feature_batch)
  font_labels.append(label_batch)

In [22]:
font_vectors_c = tf.concat(font_vectors, axis=0)
font_labels_c = tf.concat(font_labels, axis=0)

In [23]:
print(font_vectors_c.shape)
print(font_labels_c.shape)

(64, 1280)
(64,)


## Transfer Learning Tutorial

[Transfer learning with a pretrained ConvNet](https://www.tensorflow.org/tutorials/images/transfer_learning)